# WHO GHE (latest)

In [ ]:
dest_dir = "/tmp/ghe_latest"

In [ ]:
from owid import walden, catalog  # type: ignore
import tempfile
from zipfile import ZipFile
import os
import pandas as pd

from etl.steps.data.converters import convert_walden_metadata

## 1. Locate the dataset in Walden

In [ ]:
raw_dataset = walden.Catalog().find_latest("who", "ghe")

In [ ]:
raw_dataset

## 2. Extract the zip file to a temporary directory

In [ ]:
with tempfile.TemporaryDirectory() as dirname:
    pass

os.mkdir(dirname)
dirname

In [ ]:
ZipFile(raw_dataset.local_path).extractall(dirname)

In [ ]:
csv_file = os.path.join(dirname, "who_ghe", "_all_countries.csv")

## 3. Load the data frame and prune excess columns

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
df.iloc[:1].T

In [ ]:
df.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1, inplace=True)

In [ ]:
df.drop([col for col in df.columns if col.startswith("Sys_")], axis=1, inplace=True)

In [ ]:
df.drop([col for col in df.columns if col.startswith("FL_")], axis=1, inplace=True)

In [ ]:
df.columns = [col.lower() for col in df.columns]

In [ ]:
df.drop("_recordid", axis=1, inplace=True)

In [ ]:
df.iloc[0]

## 4. Save as a dataset

In [ ]:
raw_dataset

In [ ]:
ds = catalog.Dataset.create_empty(dest_dir)
ds.metadata = convert_walden_metadata(raw_dataset)
ds.save()

### Add cause codes

In [ ]:
ghe_causes = (
    df[["ghe_cause_code", "ghe_cause_title"]]
    .drop_duplicates()
    .set_index("ghe_cause_code")
)
ghe_causes = catalog.Table(ghe_causes)

In [ ]:
ghe_causes

In [ ]:
ghe_causes.metadata = catalog.TableMeta(
    short_name="ghe_causes",
    title="GHE Cause Codes",
    description="Integer codes for common GHE causes and their human readable names",
)
ds.add(ghe_causes)

### Add estimates

In [ ]:
df.drop("ghe_cause_code", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
estimates = catalog.Table(df)

In [ ]:
estimates.set_index(
    ["country_code", "year", "ghe_cause_title", "sex_code", "agegroup_code"],
    inplace=True,
)

In [ ]:
estimates.head()

In [ ]:
estimates.metadata.short_name = "estimates"
estimates.metadata.description = "GHE estimated burden of disease"

In [ ]:
ds.add(estimates)

## Cleanup

In [ ]:
import shutil

In [ ]:
shutil.rmtree(dirname)